### Chapter 25: Low Power Design Features

#### Introduction to Low Power Design

Low power design has become crucial in modern semiconductor design due to:
- Battery-powered devices requiring extended operation
- Thermal management in high-performance processors
- Energy efficiency regulations and environmental concerns
- Cost reduction through lower power consumption

SystemVerilog provides several features and methodologies to support low power design verification and implementation.

#### Power-Aware Simulation

Power-aware simulation enables verification of power management features during functional verification.

##### Basic Power-Aware Constructs

```systemverilog
// Power domain declaration
module cpu_core (
    input logic clk,
    input logic reset_n,
    input logic power_enable,
    // ... other signals
);

// Power state variables
logic power_on;
logic retention_mode;

// Power state control
always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n) begin
        power_on <= 1'b0;
        retention_mode <= 1'b0;
    end else begin
        power_on <= power_enable;
        retention_mode <= !power_enable && retention_signal;
    end
end

// Conditional simulation based on power state
always_comb begin
    if (!power_on) begin
        // Power-off behavior
        cpu_outputs = 'x;  // Unknown state
    end else if (retention_mode) begin
        // Retention mode - maintain critical state
        cpu_outputs = retained_values;
    end else begin
        // Normal operation
        cpu_outputs = normal_operation_outputs;
    end
end

endmodule
```

##### Power State Modeling

```systemverilog
// Power state enumeration
typedef enum logic [2:0] {
    POWER_OFF     = 3'b000,
    POWER_ON      = 3'b001,
    RETENTION     = 3'b010,
    SLEEP_LIGHT   = 3'b011,
    SLEEP_DEEP    = 3'b100
} power_state_t;

class power_manager;
    power_state_t current_state;
    power_state_t next_state;
    
    // Power transition methods
    function void request_power_down();
        case (current_state)
            POWER_ON: next_state = SLEEP_LIGHT;
            SLEEP_LIGHT: next_state = SLEEP_DEEP;
            SLEEP_DEEP: next_state = POWER_OFF;
            default: next_state = current_state;
        endcase
    endfunction
    
    function void request_power_up();
        case (current_state)
            POWER_OFF: next_state = RETENTION;
            RETENTION: next_state = POWER_ON;
            SLEEP_DEEP: next_state = SLEEP_LIGHT;
            SLEEP_LIGHT: next_state = POWER_ON;
            default: next_state = current_state;
        endcase
    endfunction
    
    // Power state transition
    task apply_power_transition();
        current_state = next_state;
        $display("Power state changed to: %s", current_state.name());
    endtask
endclass
```

#### Unified Power Format (UPF)

UPF is an IEEE standard (1801) for specifying power intent in electronic designs.

##### UPF Basic Commands

```systemverilog
// UPF commands are typically in separate .upf files
// but can be embedded in SystemVerilog for simulation

// Create power domain
create_power_domain TOP
create_power_domain CPU -elements {cpu_inst}
create_power_domain GPU -elements {gpu_inst}

// Create supply network
create_supply_net VDD -domain TOP
create_supply_net VDD_CPU -domain CPU  
create_supply_net VDD_GPU -domain GPU
create_supply_net VSS -domain TOP

// Create supply ports
create_supply_port VDD -domain TOP -direction in
create_supply_port VDD_CPU -domain CPU -direction in
create_supply_port VSS -domain TOP -direction in

// Connect supply network
connect_supply_net VDD -ports VDD
connect_supply_net VDD_CPU -ports VDD_CPU
connect_supply_net VSS -ports VSS
```

##### UPF Power States

```systemverilog
// Define power states for domains
add_power_state TOP.primary -state {
    -name ON -logic_expr {VDD == 1'b1 && VSS == 1'b0}
}

add_power_state CPU.primary -state {
    -name ON -logic_expr {VDD_CPU == 1'b1}
} -state {
    -name OFF -logic_expr {VDD_CPU == 1'b0}
} -state {
    -name RETENTION -logic_expr {VDD_CPU == 1'b0 && retention_supply == 1'b1}
}

// Power state transitions
create_power_state_group CPU_states -states {CPU.primary.ON CPU.primary.OFF CPU.primary.RETENTION}
```

##### Integrating UPF with SystemVerilog

```systemverilog
module power_aware_cpu (
    input logic clk,
    input logic reset_n,
    input logic vdd_cpu,
    input logic retention_supply,
    input logic [31:0] data_in,
    output logic [31:0] data_out
);

// Power state detection
logic power_good;
logic retention_mode;

assign power_good = vdd_cpu;
assign retention_mode = !vdd_cpu && retention_supply;

// Retention registers
logic [31:0] retention_data;

always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n) begin
        retention_data <= '0;
    end else if (power_good && !retention_mode) begin
        retention_data <= data_in;  // Store in retention during normal operation
    end
    // Retain value during retention_mode
end

// Output logic with power awareness
always_comb begin
    if (!power_good && !retention_mode) begin
        data_out = 'x;  // Unknown when powered off
    end else if (retention_mode) begin
        data_out = retention_data;  // Output retained value
    end else begin
        data_out = data_in;  // Normal operation
    end
end

endmodule
```

#### Power Domains and Islands

Power domains allow different parts of a design to be powered independently.

##### Power Domain Implementation

```systemverilog
// Power domain interface
interface power_domain_if;
    logic vdd;
    logic vss;
    logic enable;
    logic retention;
    logic isolation;
    
    modport master (
        output vdd, vss, enable, retention, isolation
    );
    
    modport slave (
        input vdd, vss, enable, retention, isolation
    );
endinterface

// Power island module
module power_island #(
    parameter DOMAIN_NAME = "DEFAULT"
)(
    power_domain_if.slave pwr_if,
    input logic clk,
    input logic reset_n,
    // Functional interfaces
    input logic [31:0] data_in,
    output logic [31:0] data_out,
    output logic valid_out
);

// Power state logic
logic domain_active;
logic retention_active;

assign domain_active = pwr_if.vdd && pwr_if.enable;
assign retention_active = pwr_if.retention && !domain_active;

// Isolation logic
logic [31:0] isolated_data_out;
logic isolated_valid_out;

always_comb begin
    if (pwr_if.isolation) begin
        isolated_data_out = '0;  // Or specific isolation value
        isolated_valid_out = 1'b0;
    end else begin
        isolated_data_out = internal_data_out;
        isolated_valid_out = internal_valid_out;
    end
end

assign data_out = isolated_data_out;
assign valid_out = isolated_valid_out;

// Internal logic with power awareness
logic [31:0] internal_data_out;
logic internal_valid_out;
logic [31:0] retention_register;

always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n) begin
        retention_register <= '0;
        internal_data_out <= '0;
        internal_valid_out <= 1'b0;
    end else if (domain_active) begin
        // Normal operation
        internal_data_out <= data_in + 1;  // Example processing
        internal_valid_out <= 1'b1;
        retention_register <= data_in;  // Save for retention
    end else if (retention_active) begin
        // Retention mode
        internal_data_out <= retention_register;
        internal_valid_out <= 1'b0;
    end else begin
        // Powered down
        internal_data_out <= 'x;
        internal_valid_out <= 1'bx;
    end
end

endmodule
```

##### Power Controller

```systemverilog
module power_controller (
    input logic clk,
    input logic reset_n,
    input logic power_request,
    input logic [2:0] target_domain,
    power_domain_if.master cpu_domain,
    power_domain_if.master gpu_domain,
    power_domain_if.master mem_domain
);

// Power sequencing state machine
typedef enum logic [3:0] {
    IDLE,
    POWER_UP_SEQ1,
    POWER_UP_SEQ2,
    POWER_UP_SEQ3,
    ACTIVE,
    POWER_DOWN_SEQ1,
    POWER_DOWN_SEQ2,
    POWER_DOWN_SEQ3,
    POWERED_DOWN
} power_seq_state_t;

power_seq_state_t current_state, next_state;

// Domain selection
power_domain_if selected_domain;

always_comb begin
    case (target_domain)
        3'b001: selected_domain = cpu_domain;
        3'b010: selected_domain = gpu_domain;
        3'b100: selected_domain = mem_domain;
        default: selected_domain = cpu_domain;
    endcase
end

// Power sequencing FSM
always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n)
        current_state <= POWERED_DOWN;
    else
        current_state <= next_state;
end

always_comb begin
    next_state = current_state;
    
    case (current_state)
        IDLE: begin
            if (power_request)
                next_state = POWER_UP_SEQ1;
        end
        
        POWER_UP_SEQ1: begin
            next_state = POWER_UP_SEQ2;  // Enable isolation
        end
        
        POWER_UP_SEQ2: begin
            next_state = POWER_UP_SEQ3;  // Apply power
        end
        
        POWER_UP_SEQ3: begin
            next_state = ACTIVE;  // Remove isolation
        end
        
        ACTIVE: begin
            if (!power_request)
                next_state = POWER_DOWN_SEQ1;
        end
        
        POWER_DOWN_SEQ1: begin
            next_state = POWER_DOWN_SEQ2;  // Enable isolation
        end
        
        POWER_DOWN_SEQ2: begin
            next_state = POWER_DOWN_SEQ3;  // Enable retention
        end
        
        POWER_DOWN_SEQ3: begin
            next_state = POWERED_DOWN;  // Remove power
        end
        
        POWERED_DOWN: begin
            if (power_request)
                next_state = IDLE;
        end
    endcase
end

// Control signal generation
always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n) begin
        selected_domain.vdd <= 1'b0;
        selected_domain.enable <= 1'b0;
        selected_domain.isolation <= 1'b1;
        selected_domain.retention <= 1'b0;
    end else begin
        case (current_state)
            POWER_UP_SEQ1: begin
                selected_domain.isolation <= 1'b1;
            end
            
            POWER_UP_SEQ2: begin
                selected_domain.vdd <= 1'b1;
                selected_domain.enable <= 1'b1;
            end
            
            POWER_UP_SEQ3: begin
                selected_domain.isolation <= 1'b0;
            end
            
            POWER_DOWN_SEQ1: begin
                selected_domain.isolation <= 1'b1;
            end
            
            POWER_DOWN_SEQ2: begin
                selected_domain.retention <= 1'b1;
            end
            
            POWER_DOWN_SEQ3: begin
                selected_domain.vdd <= 1'b0;
                selected_domain.enable <= 1'b0;
            end
            
            POWERED_DOWN: begin
                selected_domain.retention <= 1'b0;
            end
        endcase
    end
end

endmodule
```

#### Clock and Power Gating

Clock and power gating are essential techniques for reducing dynamic and static power consumption.

##### Clock Gating Implementation

```systemverilog
// Clock gating cell
module clock_gate (
    input logic clk_in,
    input logic enable,
    input logic test_enable,  // For DFT
    output logic clk_out
);

// Latch to avoid glitches
logic enable_latched;

// Latch enable signal on negative edge to avoid glitches
always_latch begin
    if (!clk_in)
        enable_latched <= enable || test_enable;
end

// Gated clock output
assign clk_out = clk_in && enable_latched;

endmodule

// Usage in a module
module cpu_with_clock_gating (
    input logic clk,
    input logic reset_n,
    input logic cpu_enable,
    input logic fpu_enable,
    input logic cache_enable,
    input logic [31:0] instruction,
    output logic [31:0] result
);

// Gated clocks
logic cpu_clk, fpu_clk, cache_clk;

// Clock gating instances
clock_gate cpu_cg (
    .clk_in(clk),
    .enable(cpu_enable),
    .test_enable(1'b0),
    .clk_out(cpu_clk)
);

clock_gate fpu_cg (
    .clk_in(clk),
    .enable(fpu_enable),
    .test_enable(1'b0),
    .clk_out(fpu_clk)
);

clock_gate cache_cg (
    .clk_in(clk),
    .enable(cache_enable),
    .test_enable(1'b0),
    .clk_out(cache_clk)
);

// Functional units using gated clocks
cpu_core cpu_inst (
    .clk(cpu_clk),
    .reset_n(reset_n),
    .instruction(instruction),
    .result(result)
);

// Additional modules would use their respective gated clocks

endmodule
```

##### Advanced Clock Gating with Power Management

```systemverilog
// Hierarchical clock gating controller
module clock_gate_controller (
    input logic clk,
    input logic reset_n,
    input logic global_enable,
    input logic [7:0] unit_active,  // Activity indicators
    input logic [7:0] force_enable, // Force enable for debug
    output logic [7:0] clock_enables
);

// Activity detection and filtering
logic [7:0] activity_filtered;
logic [3:0] idle_counter [8];

// Activity filtering to prevent unnecessary clock switching
genvar i;
generate
    for (i = 0; i < 8; i++) begin : activity_filter
        always_ff @(posedge clk or negedge reset_n) begin
            if (!reset_n) begin
                idle_counter[i] <= '0;
                activity_filtered[i] <= 1'b0;
            end else begin
                if (unit_active[i]) begin
                    idle_counter[i] <= '0;
                    activity_filtered[i] <= 1'b1;
                end else if (idle_counter[i] < 4'hF) begin
                    idle_counter[i] <= idle_counter[i] + 1;
                    if (idle_counter[i] == 4'h3) // 4 cycle delay
                        activity_filtered[i] <= 1'b0;
                end
            end
        end
    end
endgenerate

// Final clock enable generation
always_comb begin
    for (int j = 0; j < 8; j++) begin
        clock_enables[j] = global_enable && 
                          (activity_filtered[j] || force_enable[j]);
    end
end

endmodule
```

##### Power Gating Implementation

```systemverilog
// Power switch controller
module power_switch_controller (
    input logic clk,
    input logic reset_n,
    input logic power_down_req,
    input logic power_up_req,
    output logic power_switch_enable,
    output logic isolation_enable,
    output logic retention_enable,
    output logic power_good
);

typedef enum logic [2:0] {
    POWERED_ON,
    ISOLATING,
    RETAINING,
    POWERING_DOWN,
    POWERED_OFF,
    POWERING_UP,
    RESTORING
} power_state_t;

power_state_t current_state, next_state;
logic [7:0] delay_counter;

// State machine for power gating sequence
always_ff @(posedge clk or negedge reset_n) begin
    if (!reset_n) begin
        current_state <= POWERED_OFF;
        delay_counter <= '0;
    end else begin
        current_state <= next_state;
        if (next_state != current_state)
            delay_counter <= '0;
        else
            delay_counter <= delay_counter + 1;
    end
end

always_comb begin
    next_state = current_state;
    
    case (current_state)
        POWERED_ON: begin
            if (power_down_req)
                next_state = ISOLATING;
        end
        
        ISOLATING: begin
            if (delay_counter >= 8'd2)  // Wait 2 cycles
                next_state = RETAINING;
        end
        
        RETAINING: begin
            if (delay_counter >= 8'd2)
                next_state = POWERING_DOWN;
        end
        
        POWERING_DOWN: begin
            if (delay_counter >= 8'd10)  // Wait for power to stabilize
                next_state = POWERED_OFF;
        end
        
        POWERED_OFF: begin
            if (power_up_req)
                next_state = POWERING_UP;
        end
        
        POWERING_UP: begin
            if (delay_counter >= 8'd10)
                next_state = RESTORING;
        end
        
        RESTORING: begin
            if (delay_counter >= 8'd2)
                next_state = POWERED_ON;
        end
    endcase
end

// Output control signals
always_comb begin
    case (current_state)
        POWERED_ON: begin
            power_switch_enable = 1'b1;
            isolation_enable = 1'b0;
            retention_enable = 1'b0;
            power_good = 1'b1;
        end
        
        ISOLATING: begin
            power_switch_enable = 1'b1;
            isolation_enable = 1'b1;
            retention_enable = 1'b0;
            power_good = 1'b1;
        end
        
        RETAINING: begin
            power_switch_enable = 1'b1;
            isolation_enable = 1'b1;
            retention_enable = 1'b1;
            power_good = 1'b1;
        end
        
        POWERING_DOWN, POWERED_OFF: begin
            power_switch_enable = 1'b0;
            isolation_enable = 1'b1;
            retention_enable = 1'b1;
            power_good = 1'b0;
        end
        
        POWERING_UP: begin
            power_switch_enable = 1'b1;
            isolation_enable = 1'b1;
            retention_enable = 1'b1;
            power_good = 1'b0;
        end
        
        RESTORING: begin
            power_switch_enable = 1'b1;
            isolation_enable = 1'b1;
            retention_enable = 1'b0;
            power_good = 1'b1;
        end
    endcase
end

endmodule
```

#### Power-Aware Verification

##### Power-Aware Testbench

```systemverilog
class power_aware_testbench;
    
    // Virtual interfaces
    virtual power_domain_if pwr_vif;
    virtual clock_if clk_vif;
    
    // Power scenarios
    typedef enum {
        NORMAL_OPERATION,
        POWER_CYCLE,
        RETENTION_TEST,
        ISOLATION_TEST,
        CLOCK_GATING_TEST
    } power_test_scenario_t;
    
    // Test execution
    task run_power_test(power_test_scenario_t scenario);
        case (scenario)
            NORMAL_OPERATION: run_normal_operation();
            POWER_CYCLE: run_power_cycle_test();
            RETENTION_TEST: run_retention_test();
            ISOLATION_TEST: run_isolation_test();
            CLOCK_GATING_TEST: run_clock_gating_test();
        endcase
    endtask
    
    // Power cycle test
    task run_power_cycle_test();
        $display("Starting power cycle test");
        
        // Initialize with power on
        pwr_vif.vdd <= 1'b1;
        pwr_vif.enable <= 1'b1;
        pwr_vif.isolation <= 1'b0;
        
        repeat(10) @(posedge clk_vif.clk);
        
        // Power down sequence
        $display("Powering down...");
        pwr_vif.isolation <= 1'b1;
        repeat(2) @(posedge clk_vif.clk);
        
        pwr_vif.retention <= 1'b1;
        repeat(2) @(posedge clk_vif.clk);
        
        pwr_vif.vdd <= 1'b0;
        pwr_vif.enable <= 1'b0;
        repeat(10) @(posedge clk_vif.clk);
        
        // Power up sequence
        $display("Powering up...");
        pwr_vif.vdd <= 1'b1;
        pwr_vif.enable <= 1'b1;
        repeat(10) @(posedge clk_vif.clk);
        
        pwr_vif.retention <= 1'b0;
        repeat(2) @(posedge clk_vif.clk);
        
        pwr_vif.isolation <= 1'b0;
        repeat(2) @(posedge clk_vif.clk);
        
        $display("Power cycle test completed");
    endtask
    
    // Retention test
    task run_retention_test();
        logic [31:0] test_data = 32'hDEADBEEF;
        logic [31:0] read_data;
        
        $display("Starting retention test");
        
        // Write data during normal operation
        write_data(test_data);
        read_data = read_data();
        assert(read_data == test_data) else 
            $error("Data mismatch before retention");
        
        // Enter retention mode
        enter_retention_mode();
        
        // Try to read during retention
        read_data = read_data();
        assert(read_data == test_data) else 
            $error("Data lost during retention");
        
        // Exit retention mode
        exit_retention_mode();
        
        // Verify data after retention
        read_data = read_data();
        assert(read_data == test_data) else 
            $error("Data lost after retention");
        
        $display("Retention test completed");
    endtask
    
    // Helper tasks
    task write_data(input logic [31:0] data);
        // Implementation depends on DUT interface
    endtask
    
    function logic [31:0] read_data();
        // Implementation depends on DUT interface
        return 32'h0;
    endfunction
    
    task enter_retention_mode();
        pwr_vif.retention <= 1'b1;
        pwr_vif.vdd <= 1'b0;
        repeat(5) @(posedge clk_vif.clk);
    endtask
    
    task exit_retention_mode();
        pwr_vif.vdd <= 1'b1;
        repeat(5) @(posedge clk_vif.clk);
        pwr_vif.retention <= 1'b0;
        repeat(2) @(posedge clk_vif.clk);
    endtask
    
endclass
```

#### Best Practices for Low Power Design

1. **Power Planning**: Plan power domains early in the design phase
2. **Verification Strategy**: Include power-aware verification from the start
3. **Clock Gating**: Implement fine-grained clock gating for maximum power savings
4. **Retention Strategy**: Carefully select what state needs to be retained
5. **Isolation**: Properly isolate powered-down domains
6. **Power Sequencing**: Implement proper power-up/down sequences
7. **Tool Integration**: Use UPF for tool compatibility across the design flow

#### Summary

Low power design features in SystemVerilog enable:
- Power-aware simulation and verification
- UPF integration for power intent specification
- Power domain and island implementation
- Clock and power gating techniques
- Comprehensive verification of power management features

These features are essential for modern low-power semiconductor design, enabling battery-operated devices and energy-efficient systems.